In [1]:
import torch
import math

from torch import nn

In [2]:
def generate_disc_set(nb):
    # creating the circle in the middle of the points
    axis = torch.FloatTensor(1,2).uniform_(0.5,0.5)
    r = (2/math.pi)**0.5
    
    train_input = torch.FloatTensor(nb,2).uniform_(-1,1)
    train_target = torch.FloatTensor(nb, 1)
    test_input = torch.FloatTensor(nb,2).uniform_(-1,1)
    test_target = torch.FloatTensor(nb, 1)
    
    for i in range(0, len(train_input)):
        r = (2/math.pi)**0.5
        a = abs((train_input[i] - axis).pow(2).sum(1).view(-1).pow(0.5))
        b = abs((test_input[i] - axis).pow(2).sum(1).view(-1).pow(0.5))
    
        if a < r:
            train_target[i] = 1
        else:
            train_target[i] = 0
            
        if b < r:
            test_target[i] = 1
        else:
            test_target[i] = 0
        
    return train_input, train_target, test_input, test_target

In [3]:
generate_disc_set(10)

(tensor([[-0.7450,  0.0987],
         [ 0.1427,  0.7343],
         [-0.1838, -0.5525],
         [ 0.4866, -0.5477],
         [ 0.6890,  0.9855],
         [-0.6137,  0.5589],
         [-0.2322, -0.0456],
         [ 0.7438,  0.4844],
         [-0.7119, -0.0946],
         [-0.3627, -0.9845]]),
 tensor([[0.],
         [1.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.]]),
 tensor([[-0.1130, -0.1824],
         [ 0.3273, -0.8459],
         [-0.6567, -0.2711],
         [ 0.5984,  0.1175],
         [ 0.6972, -0.4499],
         [-0.7739, -0.7436],
         [ 0.0316,  0.9660],
         [-0.3546,  0.3723],
         [ 0.4880, -0.5276],
         [ 0.7739, -0.5068]]),
 tensor([[0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.]]))

In [20]:
class Module(object):
    
    def forward (self, *input) :
        raise NotImplementedError
        
    def backward (self, *gradwrtoutput ) :
        raise NotImplementedError
        
    def param(self) :
        return []
    
    
class Tanh(Module):
    def __init__(self):
        super(Tanh, self).__init__()
    
    def forward(self, x):
        self.x = x
        return x.tanh()
        
    def backward(self, grad):
        ds_dx = 4 * (self.x.exp() + self.x.mul(-1).exp()).pow(-2)
        dl_dx = ds_dx*grad
        return dl_dx
        
    def params(self):
        return []

    

class Linear(Module):
    def __init__(self, dim_in, dim_out):
        super(Linear, self).__init__()
        self.dim_in = dim_in
        self.dim_out = dim_out
        self.epsilon = 1e-3
        self.x = 0

        # Initialize weights
        self.w = xavier_normal_(torch.empty(self.dim_out, self.dim_in))
        self.b = torch.empty(self.dim_out).normal_(0, self.epsilon)

        # Initialize gradient
        self.dl_dw = torch.empty(self.w.size())
        self.dl_db = torch.empty(self.b.size())
    
    def forward(self, x):
        self.x = x
        return self.x.mm(self.w.t()) + self.b


    def backward(self, grad):
        ds_dx = self.w.t()

        # do the same for every batch (batch dim becomes 1)
        dl_dx = ds_dx @ grad.t()

        # put batch dim back to 0
        dl_dx = dl_dx.t()

        # sum over all the outer product between (grad_1 * x_1^T) (_1 denotes not using mini-batches)
        self.dl_dw.add_(grad.t() @ self.x)

        # sum over the batch
        self.dl_db.add_(grad.sum(0))

        return dl_dx
        
    def params(self):
        return [(self.w, self.b), (self.dl_dw, self.dl_db)]
    
    def update_params(self, eta):
        self.w = self.w - eta * self.dl_dw
        self.b = self.b - eta * self.dl_db
    
    
    
    
    
    
    

In [30]:
Tanh.forward(x)

TypeError: forward() missing 1 required positional argument: 'x'

In [26]:
model = Module()

In [28]:
model

TypeError: 'Module' object is not callable